# **Project: Predicting Stock Prices using Algorithms**

## **Project Aim**
There are many factors that impact the stock price of a particular company. Can we develop a machine learning model that can accurately <br>
predict the stock price of companies, in order to determine whether the stock is a good buy or not, given the factors? <br>
<br>
The stock that our project is based on will be **Microsoft (MSFT)**. Microsoft is a technology giant found in 1975 by Bill Gates and Paul Allen. <br>Microsoft is renowned for its software products, which includes the Windows operating system (OS) and the Microsoft Office suite among many others. <br>Recently, Microsoft has dipped its toes in cloud computing with Azure, as well as artifical intelligence through significant investments in OpenAI.<br> It is one of the world's most valuable companies, and is currently headed by CEO Satya Nadella.

### **Disclaimer** 
**This project is created fully for informational and educational purposes. Predicting the stock market price is a highly complex task that involves<br> great financial risks, and past performance is not indicative of future results. Please consult with a qualified financial advisor before making any<br> form of investment decisions, and I strongly discourage you from using my machine learning model as a basis for your financial decisions. <br>
<br>
Furthermore, all data gathered for this project's use is obtained from open-source websites. <br>No information used in this project is obtained through unauthorized means.**

## **Phase One: Training Data Collection**
**Timescale: January 2021 - June 2023**

In [33]:
# importing libraries
import numpy as np
import pandas as pd

# yahoo finance
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

### **1. Stock Prices**
Source: Yahoo Finance, through Python library ***yfinance***<br><br>
**Data:**
- Open: price of MSFT when the stock market opens
- High: highest daily price of MSFT
- Low: lowest daily price of MSFT
- Close: price of MSFT when the stock market closes
- Adj Close: price of MSFT when the stock market closes, adjusted to account for corporate action.
    - **Indicates true value of MSFT over time**
- Volume: total number of orders made during the day

Since we are focusing on daily predictions, we only require Date, Adj. Close and Volume of MSFT.

In [34]:
df_prices = yf.download('MSFT', start='2020-12-31', end='2023-06-30')

# filtering out columns
df_prices = df_prices.reset_index()
df_prices = df_prices[['Date','Adj Close','Volume']].rename(columns={'Adj Close':'MSFT_Close','Volume':'MSFT_Volume'})
df_prices

[*********************100%%**********************]  1 of 1 completed


,Date,MSFT_Close,MSFT_Volume
0,2020-12-31,217.042542,20942100
1,2021-01-04,212.426926,37130100
2,2021-01-05,212.631821,23823000
3,2021-01-06,207.118454,35930700
4,2021-01-07,213.012390,27694500
...,...,...,...
622,2023-06-23,334.312164,23084700
623,2023-06-26,327.905762,21520600
624,2023-06-27,333.863159,24354100
625,2023-06-28,335.140442,20259500


In [35]:
# date imputing
all_days = pd.date_range('2020-12-31', '2023-06-30')
all_days = pd.DataFrame({"Date": all_days})

df_prices = df_prices.merge(all_days, left_on='Date', right_on='Date', how='right')
df_prices = df_prices.fillna(method='ffill')
df_prices = df_prices.drop(index=0)
df_prices

,Date,MSFT_Close,MSFT_Volume
1,2021-01-01,217.042542,20942100.0
2,2021-01-02,217.042542,20942100.0
3,2021-01-03,217.042542,20942100.0
4,2021-01-04,212.426926,37130100.0
5,2021-01-05,212.631821,23823000.0
...,...,...,...
907,2023-06-26,327.905762,21520600.0
908,2023-06-27,333.863159,24354100.0
909,2023-06-28,335.140442,20259500.0
910,2023-06-29,334.342102,16997000.0


### **2. Additional Financial Data**
1. **Economic Indicators**
    - GDP Growth Rate (Quarterly)
    - Inflation Rate (CPI, Monthly)
2. **Market Indices**: NASDAQ
    - MSFT is listed in NASDAQ.
3. **Earnings Report**
    - Total Revenue
    - Earnings per Share (Basic EPS) = Net Income (Profits) / Outstanding Shares
        - Outstanding Shares: shares owned by investors and company executives
    - Free Cash Flow (FCF) = Cash from Operations (CFO) - Capital Expenditures

#### **Economic Indicators**
Sources:
- GDP Growth Rate (quarterly): https://data.oecd.org/gdp/quarterly-gdp.htm
- Inflation rate (CPI, monthly): https://data.oecd.org/price/inflation-cpi.htm

In [36]:
# Inflation Rate
df_inf = pd.read_csv('data/CPI_USA_Monthly.csv')[['TIME','Value']]
df_inf = df_inf[df_inf['TIME']<='2023-06'].rename(columns={'Value':'Inflation_Rate'})

# GDP
df_gdp = pd.read_csv('data/GDP_USA_Quarterly.csv')[['TIME','Value']].rename(columns={'Value':'GDP_Growth_Rate'})
df_gdp['TIME'] = pd.to_datetime(df_gdp['TIME']).dt.strftime('%Y-%m')

# Combined
df_econs = df_inf.merge(df_gdp, left_on='TIME', right_on='TIME', how='left')
df_econs = df_econs.fillna(method='ffill')
df_econs['TIME'] = pd.to_datetime(df_econs['TIME'])

# Date Imputing
df_econs = all_days.merge(df_econs, left_on='Date', right_on='TIME', how='left')
df_econs = df_econs.drop(index=0,columns='TIME').fillna(method='ffill')
df_econs

,Date,Inflation_Rate,GDP_Growth_Rate
1,2021-01-01,1.399770,1.542792
2,2021-01-02,1.399770,1.542792
3,2021-01-03,1.399770,1.542792
4,2021-01-04,1.399770,1.542792
5,2021-01-05,1.399770,1.542792
...,...,...,...
907,2023-06-26,2.969178,0.496812
908,2023-06-27,2.969178,0.496812
909,2023-06-28,2.969178,0.496812
910,2023-06-29,2.969178,0.496812


#### **Market Indices**
- MSFT is listed under **NASDAQ**, hence we will be using the **NASDAQ Composite Index (.IXIC)**.
- All data in this section is obtained from Yahoo Finance, accessed through the Python library ***yfinance***.

In [37]:
# downloading data
nasdaq = yf.Ticker('^IXIC').history(period='max').reset_index()
nasdaq['Date'] = pd.to_datetime(nasdaq['Date'].dt.strftime('%Y-%m-%d'))
nasdaq = nasdaq[nasdaq['Date']>='2020-12-31']

# date imputing
nasdaq = all_days.merge(nasdaq, left_on='Date', right_on='Date', how='left')
nasdaq = nasdaq.fillna(method='ffill')
nasdaq = nasdaq.drop(index=0)
nasdaq = nasdaq[['Date','Close','Volume']].rename(columns={'Close':'NASDAQ_Close','Volume':'NASDAQ_Volume'})
nasdaq

,Date,NASDAQ_Close,NASDAQ_Volume
1,2021-01-01,12888.280273,4.815840e+09
2,2021-01-02,12888.280273,4.815840e+09
3,2021-01-03,12888.280273,4.815840e+09
4,2021-01-04,12698.450195,6.636170e+09
5,2021-01-05,12818.959961,6.971860e+09
...,...,...,...
907,2023-06-26,13335.780273,4.430600e+09
908,2023-06-27,13555.669922,5.053660e+09
909,2023-06-28,13591.750000,4.533270e+09
910,2023-06-29,13591.330078,4.388140e+09


#### **Earnings Report**
Sources: 
- https://www.microsoft.com/en-us/investor/earnings/FY-2023-Q3/press-release-webcast
- https://www.macrotrends.net/stocks/charts/MSFT/microsoft/free-cash-flow

In [38]:
# Revenue and EPS
df_earnings = pd.read_excel('data/QuarterlyIncomeStatementsFY23_Modified.xlsx')
df_earnings = df_earnings.drop(columns=['Unnamed: 1','Unnamed: 2']).rename(columns={'Unnamed: 0':'Column'})
df_earnings = df_earnings[(df_earnings['Column']=='Quarter')|(df_earnings['Column']=='Total revenue')|(df_earnings['Column']=='Basic')]
df_earnings = pd.DataFrame({
    'Quarter':df_earnings.loc[0].values,
    'Revenue':df_earnings.loc[4].values,
    'EPS':df_earnings.loc[20].values
})
df_earnings = df_earnings.loc[21:30]

# Quarter to Date
quarter_to_month = {'Q1':'1','Q2':'4','Q3':'7','Q4':'10'}
df_earnings['Year'] = df_earnings['Quarter'].str[-2:]
df_earnings['Quarter'] = df_earnings['Quarter'].str[:2]
df_earnings['Month'] = df_earnings['Quarter'].map(quarter_to_month)
df_earnings['Quarter'] = pd.to_datetime(df_earnings['Year'] + df_earnings['Month'], format='%y%m')
df_earnings = df_earnings.drop(columns=['Year','Month']).rename(columns={'Quarter':'Date'})

# Revenue in millions
df_earnings['Revenue'] = df_earnings['Revenue'] * 1e6

# EPS
df_earnings['EPS'] = df_earnings['EPS'].str[1:].astype(float)

# date imputing
df_earnings = all_days.merge(df_earnings, left_on='Date', right_on='Date', how='left')
df_earnings = df_earnings.fillna(method='ffill')
df_earnings = df_earnings.drop(index=0)
df_earnings

,Date,Revenue,EPS
1,2021-01-01,3.715400e+10,1.84
2,2021-01-02,3.715400e+10,1.84
3,2021-01-03,3.715400e+10,1.84
4,2021-01-04,3.715400e+10,1.84
5,2021-01-05,3.715400e+10,1.84
...,...,...,...
907,2023-06-26,5.274700e+10,2.20
908,2023-06-27,5.274700e+10,2.20
909,2023-06-28,5.274700e+10,2.20
910,2023-06-29,5.274700e+10,2.20


In [39]:
# FCF in millions
df_cash = pd.read_excel('data/QuarterlyCashFlowStatementFY23_Modified.xlsx')
df_cash['FCF'] = df_cash['FCF'] * 1e6

# date imputing
df_cash = all_days.merge(df_cash, left_on='Date', right_on='Date', how='left')
df_cash = df_cash.fillna(method='ffill')
df_cash = df_cash.drop(index=0)

# merging with earnings
df_earnings = df_cash.merge(df_earnings, left_on='Date', right_on='Date', how='left')
df_earnings

,Date,FCF,Revenue,EPS
0,2021-01-01,2.277000e+10,3.715400e+10,1.84
1,2021-01-02,2.277000e+10,3.715400e+10,1.84
2,2021-01-03,2.277000e+10,3.715400e+10,1.84
3,2021-01-04,2.277000e+10,3.715400e+10,1.84
4,2021-01-05,2.277000e+10,3.715400e+10,1.84
...,...,...,...,...
906,2023-06-26,3.964800e+10,5.274700e+10,2.20
907,2023-06-27,3.964800e+10,5.274700e+10,2.20
908,2023-06-28,3.964800e+10,5.274700e+10,2.20
909,2023-06-29,3.964800e+10,5.274700e+10,2.20


### **3. Combining All Information**

In [40]:
df = pd.merge(df_econs, nasdaq, on='Date', how='outer')
df = pd.merge(df, df_earnings, on='Date', how='outer')
df = pd.merge(df, df_prices, on='Date', how='outer')
df = df[['Date','GDP_Growth_Rate','Inflation_Rate','NASDAQ_Close','NASDAQ_Volume','Revenue','EPS','FCF','MSFT_Volume','MSFT_Close']]
df

,Date,GDP_Growth_Rate,Inflation_Rate,NASDAQ_Close,NASDAQ_Volume,Revenue,EPS,FCF,MSFT_Volume,MSFT_Close
0,2021-01-01,1.542792,1.399770,12888.280273,4.815840e+09,3.715400e+10,1.84,2.277000e+10,20942100.0,217.042542
1,2021-01-02,1.542792,1.399770,12888.280273,4.815840e+09,3.715400e+10,1.84,2.277000e+10,20942100.0,217.042542
2,2021-01-03,1.542792,1.399770,12888.280273,4.815840e+09,3.715400e+10,1.84,2.277000e+10,20942100.0,217.042542
3,2021-01-04,1.542792,1.399770,12698.450195,6.636170e+09,3.715400e+10,1.84,2.277000e+10,37130100.0,212.426926
4,2021-01-05,1.542792,1.399770,12818.959961,6.971860e+09,3.715400e+10,1.84,2.277000e+10,23823000.0,212.631821
...,...,...,...,...,...,...,...,...,...,...
906,2023-06-26,0.496812,2.969178,13335.780273,4.430600e+09,5.274700e+10,2.20,3.964800e+10,21520600.0,327.905762
907,2023-06-27,0.496812,2.969178,13555.669922,5.053660e+09,5.274700e+10,2.20,3.964800e+10,24354100.0,333.863159
908,2023-06-28,0.496812,2.969178,13591.750000,4.533270e+09,5.274700e+10,2.20,3.964800e+10,20259500.0,335.140442
909,2023-06-29,0.496812,2.969178,13591.330078,4.388140e+09,5.274700e+10,2.20,3.964800e+10,16997000.0,334.342102


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             911 non-null    datetime64[ns]
 1   GDP_Growth_Rate  911 non-null    float64       
 2   Inflation_Rate   911 non-null    float64       
 3   NASDAQ_Close     911 non-null    float64       
 4   NASDAQ_Volume    911 non-null    float64       
 5   Revenue          911 non-null    float64       
 6   EPS              911 non-null    float64       
 7   FCF              911 non-null    float64       
 8   MSFT_Volume      911 non-null    float64       
 9   MSFT_Close       911 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 71.3 KB


We have completed the data collection phase of the project.<br>
We will be saving this dataframe into a .csv file.

In [42]:
df.to_csv('data/MSFT_data.csv', index=False)